# One Stop Stock Analyser

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np

### Input your ticker symbol here

In [ ]:
ticker = input(); ticker = ticker.upper()
stock = yf.Ticker(ticker)

logo_url = stock.info['logo_url']
Image.open(requests.get(logo_url, stream=True).raw)

## Company Summary

In [ ]:
company_summary = stock.info['longBusinessSummary']
print(company_summary, '\n')

website_url = stock.info['website']
print('Website: {} \n'.format(website_url))

sector = stock.info['sector']
print('Sector: {} \n'.format(sector))

industry = stock.info['industry']
print('Industry: {} \n'.format(industry))

market_capital = stock.info['marketCap']
print('Market Capital: {} Billions \n'.format(market_capital / 1000000000))

low52 = stock.info['fiftyTwoWeekLow']
print('52-Low: {} \n'.format(low52))

high52 = stock.info['fiftyTwoWeekHigh']
print('52-High: {} \n'.format(high52))

peg_ratio = stock.info['pegRatio']
print('Peg Ratio: {} \n'.format(round(peg_ratio, 2)))

price_to_earning_ratio = stock.info['trailingPE']
print('P/E Ratio: {} \n'.format(round(price_to_earning_ratio, 2)))

price_to_sales_ratio = stock.info['priceToSalesTrailing12Months']
print('P/S Ratio: {} \n'.format(round(price_to_sales_ratio, 2)))

current_price = stock.info['currentPrice']
print('Current Price: {} \n'.format(round(current_price, 2)))

down_from_ath = ((current_price / high52) - 1) * 100
print('Down From ATH: {}% \n'.format(round(down_from_ath, 2)))

## Profitability

In [ ]:
gross_margins = stock.info['grossMargins']
print('Gross Margins: {}% \n'.format( round(gross_margins * 100, 2)))

profit_margins = stock.info['profitMargins']
print('Profit Margins: {}% \n'.format( round(profit_margins * 100, 2)))

operating_margins = stock.info['operatingMargins']
print('Operating Margins: {}% \n'.format( round(operating_margins * 100, 2)))

return_on_assets = stock.info['returnOnAssets']
print('Return On Assets: {}% \n'.format( round(return_on_assets * 100, 2)))

return_on_equity = stock.info['returnOnEquity']
print('Return On Equity: {}% \n'.format( round(return_on_equity * 100, 2)))

ebitda_margin= stock.info['ebitda'] / stock.info['totalRevenue']
print('Ebitda Margin: {}% \n'.format( round(ebitda_margin * 100, 2)))

## Financial Health

In [ ]:
quick_ratio = stock.info['quickRatio']
print('Quick Ratio -> (Total Asset - Inventory) / Total Liability')
print('Quick Ratio: {} \n'.format( round(quick_ratio, 2)))

debt_to_equity_ratio = stock.info['debtToEquity'] / 100
print('Debt to Equity Ratio -> How much debt a company is using to finance its assets relative to the value of shareholders\' equity.')
print('Debt to Equity Ratio: {} \n'.format( round(debt_to_equity_ratio, 2)))

balancesheet = stock.get_balancesheet(freq='quarterly')
long_term_debt = balancesheet.T['Long Term Debt'][0]
total_equity = balancesheet.T['Total Stockholder Equity'][0] + balancesheet.T['Other Stockholder Equity'][0]
long_term_debt_to_equity_ratio = long_term_debt / total_equity
print('Long Term Debt to Equity Ratio: {} \n'.format( round(long_term_debt_to_equity_ratio, 2)))

## Growth

In [ ]:
revenue_growth = stock.info['revenueGrowth']
print('Revenue Growth: {}% \n'.format( round(revenue_growth * 100, 2)))

earnings_growth = stock.info['earningsQuarterlyGrowth']
print('Earnings Growth: {}% \n'.format( round(earnings_growth * 100, 2)))

## Intrinsic Value

In [ ]:
# cashflowstatement = stock.get_cashflow() # used last fiscal year
# operating_cash_flow = cashflowstatement.T['Total Cash From Operating Activities'][0]
operating_cash_flow = stock.info['operatingCashflow'] # used ttm 

# balancesheet = stock.get_balancesheet(freq='quarterly')
# cash_and_short_term_investment = balancesheet.T['Cash'][0] + balancesheet.T['Short Term Investments'][0]
cash_and_short_term_investment = stock.info['totalCash']
# short_term_and_long_term_debt = balancesheet.T['Total Current Liabilities'][0] + balancesheet.T['Long Term Debt'][0]
short_term_and_long_term_debt = stock.info['totalDebt']

last_closing_price = stock.info['previousClose']

no_shares_outstanding = stock.info['sharesOutstanding']

current_year = datetime.date.today().year

# Alter yourself
eps_growth_1_5y = int(input()) / 100
eps_growth_6_10y = 10 / 100
eps_growth_11_20y = 4.18 / 100
# Creates a list of eps growth 
eps_growth_list = [ [eps_growth_1_5y] * 5 + [eps_growth_6_10y] * 5 + [eps_growth_11_20y] * 10 ]
# Flattens the list of list
eps_growth_list = [element for sublist in eps_growth_list for element in sublist]

beta = stock.info['beta']
discount_rate = 0

# Identify the discount rate with volatility(beta) of the stock
if beta < 0.8:
    discount_rate = 4.6 / 100
elif beta < 1:
    discount_rate = 5.6 / 100
elif beta < 1.1:
    discount_rate = 6.1 / 100
elif beta < 1.2:
    discount_rate = 6.6 / 100
elif beta < 1.3:
    discount_rate = 7.1 / 100
elif beta < 1.4:
    discount_rate = 7.6 / 100
elif beta < 1.5:
    discount_rate = 8.1 / 100
else:
    discount_rate = 8.6 / 100
    
# print('Operating Cash Flow: ', operating_cash_flow)
# print('Cash and Short Term Investment: ', cash_and_short_term_investment)
# print('Short Term and Long Term Debt: ', short_term_and_long_term_debt)
# print('No. Shares Outstanding: ', no_shares_outstanding)
# print('Discount Rate: ', discount_rate)

# Initialize all parameters due to some different equation afterward
initial_projected_year = current_year + 1
initial_projected_operating_cash_flow = operating_cash_flow * (1 + eps_growth_list[0])
initial_discount_factor = 1 / (1 + discount_rate)
initial_discounted_value = ( initial_projected_operating_cash_flow * initial_discount_factor )

# Appends all initialized parameters to the list for later access
projected_year_list = [initial_projected_year]
projected_operating_cash_flow_list = [initial_projected_operating_cash_flow]
discount_factor_list = [initial_discount_factor]
discounted_value_list = [initial_discounted_value]

# A 20 years discounted cash flow calculator
for idx, projected_year in enumerate(range(1, 20)):
    
    year = projected_year_list[-1] + 1
    projected_year_list.append(year)
    
    projected_operating_cash_flow = projected_operating_cash_flow_list[-1] * (1 + eps_growth_list[1 + idx])
    projected_operating_cash_flow_list.append(projected_operating_cash_flow)
    
    discount_factor = discount_factor_list[-1] / (1 + discount_rate)
    discount_factor_list.append(discount_factor)
    
    discounted_value = projected_operating_cash_flow * discount_factor
    discounted_value_list.append(discounted_value)

# Merge into pandas dataframe
projected_df = pd.DataFrame({'Projected Year': projected_year_list,
                             'Projected Operating Cash Flow': projected_operating_cash_flow_list,
                             'Discount Factor': discount_factor_list,
                             'Discounted Value': discounted_value_list})

display(projected_df)

total_projected_cash_flow = round( sum(discounted_value_list), 2)

intrinsic_value_before_cash_debt = round( total_projected_cash_flow / no_shares_outstanding, 2)

cash_per_share = round( cash_and_short_term_investment / no_shares_outstanding, 2)

debt_per_share = round( short_term_and_long_term_debt / no_shares_outstanding, 2)

intrinsic_value_per_share = round( intrinsic_value_before_cash_debt + cash_per_share - debt_per_share, 2)

discount_or_premium = round( (last_closing_price - intrinsic_value_per_share) / intrinsic_value_per_share * 100, 2)

print('Total Projected Cash Flow: ', total_projected_cash_flow)
print('Intrinsic Value Before Cash & Debt: ', intrinsic_value_before_cash_debt)
print('Cash Per Share: ', cash_per_share)
print('Debt Per Share: ', debt_per_share)
print('Intrinsic Value Per Share: ', intrinsic_value_per_share)
print('Currently (Discount)/Premium: {}%'.format(discount_or_premium))

## Margin of Safety

In [ ]:
safety_rate = 20 / 100
bargain_price = round(intrinsic_value_per_share * (1 - safety_rate), 2)
percent_undervalue = round((last_closing_price - bargain_price) / bargain_price * 100, 2)
print('Bargain Price for {} is at {}, currently {}, {}% (safer)/riskier'.format(ticker, bargain_price, last_closing_price, percent_undervalue))

## Historical Price

In [ ]:
df = stock.history(period='5y', actions=False)['Close'].to_frame()
df['MA50'] = df['Close'].rolling(window=50, min_periods=1).mean()
df['MA150'] = df['Close'].rolling(window=150, min_periods=1).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=1).mean()
fig, ax = plt.subplots(figsize=(12, 6), dpi=150)
ax.grid()
ax.plot(df.index, df.Close, label='Close Price')
ax.plot(df.index, df.MA50, label='50-day MA')
ax.plot(df.index, df.MA150, label='150-day MA')
ax.plot(df.index, df.MA200, label='200-day MA')
ax.set_xlabel('Date')
ax.set_ylabel('Price ($)')
ax.set_title('{} 5-Year History Price'.format(ticker))
ax.legend()
plt.show()

## Signal

In [ ]:
df['signal'] = 0.0
df['signal'] = np.where(df['MA50'] > df['MA200'], 1.0, 0.0)
df['position'] = df['signal'].diff()

fig, ax = plt.subplots(figsize=(12, 6), dpi=150)
ax.grid()
ax.plot(df.index, df.Close, label='Close Price')
ax.plot(df.index, df.MA50, label='50-day MA')
ax.plot(df.index, df.MA150, label='150-day MA')
ax.plot(df.index, df.MA200, label='200-day MA')
ax.plot(df[df['position'] == 1].index, df['MA50'][df['position'] == 1], '^', markersize=10, color='g', label='buy')
ax.plot(df[df['position'] == -1].index, df['MA50'][df['position'] == -1], 'v', markersize=10, color='r', label='sell')
ax.set_xlabel('Date')
ax.set_ylabel('Price ($)')
ax.set_title('{} Buying and Selling Signal'.format(ticker))
ax.legend()
plt.show()